In [ ]:
"""return a tuple with recall, precision, and f1 for one example"""

def computeF1(goldList, predictedList):
    """Assume all questions have at least one answer"""
    if len(goldList) == 0:
        if len(predictedList) == 0:
            return (1, 1, 1)
        else:
            return (0, 0, 0)
    """If we return an empty list recall is zero and precision is one"""
    if len(predictedList) == 0:
        return (0, 1, 0)
    """It is guaranteed now that both lists are not empty"""
    #goldList = parse_result_list(goldList)
    #predictedList = parse_result_list(predictedList)

    precision = 0
    for entity in predictedList:
        if entity in goldList:
            precision += 1

    precision = float(precision) / len(predictedList)

    recall = 0
    for entity in goldList:
        if entity in predictedList:
            recall += 1

    recall = float(recall) / len(goldList)

    f1 = 0
    if precision + recall > 0:
        f1 = 2 * recall * precision / (precision + recall)

    return (recall, precision, f1)

In [ ]:
def get_answer(question):
    """extract unique answers from dataset."""
    GT = set()
    for answer in question["answer"]:
        GT.add(answer)
    return list(GT)

In [ ]:
#把问题的标准答案整理成一个以问题Id为key的字典，方便后续使用
def get_groundtruth(corpus):
    ques_ans = {}
    with open(corpus, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
        for item in data["train"] + data["dev"] + data["test"]:
            ques_id = str(item['id'])
            #这个地方的答案需要是list类型
            ques_ans[ques_id] = {'question': item['question'], 'answer': item['answer']}
    return ques_ans

In [ ]:
"""Go over all lines and compute recall, precision and F1"""

def metric():
    averageRecall = 0
    averagePrecision = 0
    averageF1 = 0
    nCorrect = 0
    count = 0

    #这是你的数据集
    corpus = 'data.json'
    gt = get_groundtruth(corpus)
    #这里是你的QA输出结果格式也是{'train': [{'id': , 'result':}, {'id': , 'result':}], 'dev':} 和data.json类似，只不过每个问题只有id和result
    inputfile = "output.json"

    output = {}
    with open(inputfile, "r", encoding="utf-8") as f:
        outputlist = json.load(f)
        for item in outputlist:
            result = item['result']
            gold = gt[str(item['id'])]
            recall, precision, f1 = computeF1(gold, result)

            if f1 == 1:
                nCorrect += 1
            averageRecall += recall
            averagePrecision += precision
            averageF1 += f1
            count += 1

    """Print final results"""
    averageRecall = float(averageRecall) / count
    averagePrecision = float(averagePrecision) / count
    averageF1 = float(averageF1) / count
    accuracy = float(nCorrect) / count
    print("Number of questions: " + str(count))
    print("Average recall over questions: " + str(averageRecall))
    print("Average precision over questions: " + str(averagePrecision))
    print("Average f1 over questions: " + str(averageF1))
    print("Accuracy over questions: " + str(accuracy))
    averageNewF1 = 2 * averageRecall * averagePrecision / (averagePrecision + averageRecall)
    print("F1 of average recall and average precision: " + str(averageNewF1))